# This tutorial was adapted from Cholett (2021)
## You will build a CNN using transfer learning and data augmentation. The data images have already been split into training, validataion, and test sets.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import backend
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from IPython.display import display # Library to help view images
from PIL import Image # Library to help view images
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Library for data augmentation
from tensorflow.keras.preprocessing import image
import os, shutil # Library for navigating files
from google.colab import drive # Library to mount google drives
np.random.seed(1)

### We need to mount the google drive to access the images. Paste the authorization code into your browser

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
# Specify the base directory where images are located.  You need to access your data here.
base_dir = '/content/gdrive/My Drive/Cats_and_Dogs'

In [ ]:
# Specify the traning, validation, and test dirrectories.  
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# Specify the the classess in the training, validataion, and test dirrectories
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')


In [ ]:
# Check the number of files in each class dirrectory
print(len(os.listdir(train_cats_dir)))
print(len(os.listdir(train_dogs_dir)))
print(len(os.listdir(validation_cats_dir)))
print(len(os.listdir(validation_dogs_dir)))
print(len(os.listdir(test_cats_dir)))
print(len(os.listdir(test_dogs_dir)))

In [ ]:
# We need to normalize the pixels in the images.  The data will 'flow' through this generator.
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Since the file images are in a directory we need to move them from the directory into the model.  
# Keras as a function that makes this easy. Documentaion is here: https://keras.io/preprocessing/image/

train_generator = train_datagen.flow_from_directory(
    train_dir, # The directory where the train data is located
    target_size=(150, 150), # Reshape the image to 150 by 150 pixels. This is important because it makes sure all images are the same size.
    batch_size=20, # We will take images in batches of 20.
    class_mode='binary') # The classification is binary.

validataion_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

In [ ]:
# Build a plotting function
def plot_history():
  history_dict = history.history
  loss_values = history_dict['loss']
  val_loss_values = history_dict['val_loss']
  acc_values = history_dict['accuracy']
  val_acc_values = history_dict['val_accuracy']
  epochs = range(1, len(history_dict['accuracy']) + 1)

  plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
  plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

  plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
  plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  return plt.show()

In [ ]:
# Build a model
def Base_CNN():
  backend.clear_session()
  model = models.Sequential()

  model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150, 150, 3)))
  model.add(layers.MaxPool2D((2,2)))
  model.add(BatchNormalization())

  model.add(layers.Conv2D(32, (3,3), activation = 'relu'))
  model.add(layers.MaxPool2D((2,2)))
  model.add(BatchNormalization())

  model.add(layers.Conv2D(32, (3,3), activation = 'relu'))
  model.add(layers.MaxPool2D((2,2)))
  model.add(BatchNormalization())

  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer = 'rmsprop',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
model = Base_CNN()

In [ ]:
history = model.fit( # The image data must come from the image generator that takes the images from the correct dirrectory. https://keras.io/models/sequential/
    train_generator, # Images are taken from the train_generator
    steps_per_epoch=100, # The number of steps from the train_generator before one epoch if finished.  
                         # 100 steps * 20 batch size in train generator = 2000 training images per epoch. This way each traning image will be sampled once per epoch.
    epochs=50, # Train data for 50 epochs
    validation_data=validataion_generator, # Take data from the validataion generator
    validation_steps=50, # 50 steps * 20 batch size in validation generator = 1000 validation images per epoch
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)]) # We will not use call backs to stop early.

plot_history() # Use our plot function to plot the loss and accuracy.

test_loss, test_acc =model.evaluate(test_generator, steps = 50) # Test images are in a dirrectory so they must flow from dirrectory. 
                                                                           # 50 steps * 20 batch size in test generator = 1000 test images per epoch
print('test_acc:', test_acc)

model.save('cats_and_dogs_base.h5')

### The above model came out with about 70% accuracy with severe overfitting. Which is pretty good considering we only used 2000 training images!  Now lets improve using data augmentation.
#### Data augmentation allows us to randomally transform images before sending them to the model for training.  The random transformation changes the images into 'new' images and allows for an increase in traning data without have additional images. https://keras.io/preprocessing/image/ 

---



In [ ]:
datagen = ImageDataGenerator( # The image augmentaion function in Keras
    rotation_range=40, # Rotate the images randomly by 40 degrees
    width_shift_range=0.2, # Shift the image horizontally by 20%
    height_shift_range=0.2, # Shift the image veritcally by 20%
    shear_range=0.2, # Shear image by 20%
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, # Flip image horizontally 
    fill_mode='nearest') # How to fill missing pixels after a augmentaion opperation

In [ ]:
# Lets see the image augmentaion
from tensorflow.keras.preprocessing import image
img = image.load_img(os.path.join(train_cats_dir, os.listdir(train_cats_dir)[1]), target_size=(150,150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0 
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break
plt.show()

In [ ]:
# Apply the data augmentation to our data.
train_datagen2 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen2 = ImageDataGenerator(rescale=1./255) #Never apply data augmentation to test data.  You only want to normalize and resize test data. 

train_generator2 = train_datagen2.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validataion_generator2 = train_datagen2.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator2 = test_datagen2.flow_from_directory( # Resize test data
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

In [ ]:
history = model.fit(
    train_generator2,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validataion_generator2,
    validation_steps=50,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])


plot_history()

test_loss, test_acc = model.evaluate(test_generator2, steps = 50)
print('test_acc:', test_acc)

model.save('cats_and_dogs_base_augment.h5')

## An inprovment, but not a suprise. Having more data helps our accuracy. 
### But why go through the hassle of building our own CNN when there are many networks that have used powerful GPUs to classify images? We can use the weights of their models and apply them to something as simple as classiying a dog and cat.  We will use weights of the VGG16 CNN that was trained using ImageNet data.  https://keras.io/applications/

In [ ]:
from keras.applications import VGG16 # Import the VGG16 library. 

In [ ]:
backend.clear_session()
conv_base = VGG16 (weights = 'imagenet', #Useing the VGG66 CNN that was trained on ImageNet data.  
                  include_top = False, # We are using our own classification (dog or cat) and not the ImageNet multiclassification. So include top = false.
                  input_shape = (150, 150, 3))

In [ ]:
conv_base.summary() # View the VGG16 model architecture.

### The output shape of the data after the VGG16 weights are applied is (4, 4, 512).  Remember it started as (150, 150, 3).  This is from the 2D convolutions and maxpooling steps.  There are also over 14 million parameters to train.  This is way too many for the Google GPU. And we do not want to retrain the weights of the VGG16, that defeats the whole purpose of transfer learning.  Instead we will freeze the VGG16 weights and add a dense layer at the end.  

In [ ]:
conv_base.trainable = False # Freeze the VGG16 weights.

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(conv_base)

In [ ]:
conv_base.summary()# Lets look at the parameters now.

### Now the trainable parameters are 0.  We will only use the pretraind VGG16 weights.

In [ ]:
def modelvgg():
  backend.clear_session()
  modelvgg16 = models.Sequential()
  modelvgg16.add(conv_base) # Add the VG166 weights
  modelvgg16.add(layers.Flatten())
  modelvgg16.add(layers.Dense(512, activation = 'relu'))
  modelvgg16.add(layers.Dense(1, activation = 'sigmoid'))

  modelvgg16.compile(optimizer = 'rmsprop',
                      loss = 'binary_crossentropy',
                      metrics = ['accuracy'])


  return modelvgg16

In [ ]:
model = modelvgg()

In [ ]:
plot_model(model)

In [ ]:
# We will still use data augmentation
history = model.fit(
    train_generator2,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validataion_generator2,
    validation_steps=50,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])

plot_history()

test_loss, test_acc = model.evaluate(test_generator2, steps = 50)
print('test_acc:', test_acc)

model.save('cats_and_dogs_vgg.h5')

### Our model keeps getting better. We might want to add a few 2D convolutions and max pooling layers before the dense layer, or we can train the last three 2D convolution layers and maxpooling layer of the VGG16 model. We must be stratigic when choosing which layers to train in the transfer model.  We want to train convolution, max pooling, batch normalization, etc., layers that are part of block.  It would not make sense to only train max pooling or batch normalization layers without also training the convolution layers.

In [ ]:
# Now we can freeze all the VGG weights except the last few, and train those before adding it to our dense layer.
backend.clear_session()
vgg16_base_2 = VGG16(weights = 'imagenet', include_top = False, input_shape = (150, 150, 3))

# Here we freeze all the layers except the last 2.
for layer in vgg16_base_2.layers[:-2]:
  layer.trainable = False
for layer in vgg16_base_2.layers:
  print(layer, layer.trainable)

In [ ]:
vgg16_base_2.summary()

In [ ]:
plot_model(vgg16_base_2)

In [ ]:
def model_vgg_train():
  backend.clear_session()
  modelvgg16_train = models.Sequential()
  modelvgg16_train.add(vgg16_base_2)
  modelvgg16_train.add(layers.Flatten())
  modelvgg16_train.add(layers.Dense(512, activation = 'relu'))
  modelvgg16_train.add(layers.Dense(1, activation = 'sigmoid'))

  modelvgg16_train.compile('rmsprop',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

  return modelvgg16_train

In [ ]:
model = model_vgg_train()

In [ ]:
history = model.fit(
    train_generator2,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validataion_generator2,
    validation_steps=50,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])


plot_history()

test_loss, test_acc = model.evaluate(test_generator2, steps = 50)
print('test_acc:', test_acc)

model.save('cats_and_dogs_small_vgg_train.h5')

### The best model yet!

# Your Turn
### Build and optimize another model. Use weights from a different pretrained network (ie, ResNet, Inception, etc. not VGG. https://keras.io/applications/) from the Keras library. Optimize the model by adding additional layers, regularization, change activaction, adjust data augmentation etc.